In [ ]:
import sys, os
import numpy as np
np.set_printoptions(threshold=np.inf)
import pandas as pd
from utils.utils import read_txt, read_txt_np_int
from functools import lru_cache
from math import ceil, floor
from copy import deepcopy

# INPUT

In [ ]:
inputfilename = './inputs/day23.txt'

inputdata = read_txt(inputfilename)

testdata_small = ["kh-tc",
"qp-kh",
"de-cg",
"ka-co",
"yn-aq",
"qp-ub",
"cg-tb",
"vc-aq",
"tb-ka",
"wh-tc",
"yn-cg",
"kh-ub",
"ta-co",
"de-co",
"tc-td",
"tb-wq",
"wh-td",
"ta-ka",
"td-qp",
"aq-cg",
"wq-ub",
"ub-vc",
"de-ta",
"wq-aq",
"wq-vc",
"wh-yn",
"ka-de",
"kh-ta",
"co-tc",
"wh-qp",
"tb-vc",
"td-yn"]

In [ ]:
data_to_use = inputdata

connections = set([tuple(line.split('-')) for line in data_to_use])

connections

## PART 1

In [ ]:
connected_computers = {}

for link in connections:
    try:
        connected_computers[link[0]].add(link[1])
    except KeyError:
        connected_computers[link[0]] = set([link[1]])
    try:
        connected_computers[link[1]].add(link[0])
    except KeyError:
        connected_computers[link[1]] = set([link[0]])

computer_list = connected_computers.keys()


# Build 3-meshes
visited_computers = set()

three_mesh_computers = set()
for computer in computer_list:
    visited_computers.add(computer)
    for neighbor in [neigh for neigh in connected_computers[computer] if neigh not in visited_computers]:
        common_third_neighbor_list = set([third for third in connected_computers[computer].intersection(connected_computers[neighbor]) if third not in visited_computers])
        three_mesh_computers = three_mesh_computers.union(set([tuple(sorted([computer, neighbor, third_neighbor])) for third_neighbor in common_third_neighbor_list]))

# Get 3-meshes with t* computer
count_t = 0
for three_mesh in three_mesh_computers:
    if any([computer[0] == 't' for computer in three_mesh]):
        count_t += 1
        print(three_mesh)

print(f"Total count of three meshes: {len(three_mesh_computers)}")
print(f"...containing t: {count_t}")

## PART 2

In [ ]:
@lru_cache
def find_largest_party(current_mesh, common_neighbor_list, visited_computers):
    if len(common_neighbor_list) == 0:
        return current_mesh
    mesh_list = []
    for next_neighbor in common_neighbor_list:
        if next_neighbor in visited_computers:
            continue
        next_neighbor_list = (set(common_neighbor_list) & connected_computers[next_neighbor]) - set(visited_computers) # Intersection but not repeating already visited computers
        next_mesh = set(current_mesh) | {next_neighbor}  # New element in set
        #print(f"Searching mesh {next_mesh} with common neighbors {len(next_neighbor_list)}")
        mesh_list.append(find_largest_party(tuple(sorted(next_mesh)), tuple(sorted(next_neighbor_list)), visited_computers))
    return max(mesh_list, key=len)


visited_computers = set()
print(f"Number of computers: {len(computer_list)}")
for starting_computer in sorted(computer_list):
#for starting_computer in {'as', 'co'}:    
    print(f"Exploring computer: {starting_computer}")
    visited_computers.add(starting_computer)
    print(f"\tVisited computers: {sorted(visited_computers)}")
    largest_mesh = find_largest_party(tuple([starting_computer]), tuple(sorted(set(connected_computers[starting_computer]) - visited_computers)), tuple(visited_computers))
    print(f"\tLargest mesh found: {largest_mesh} with length {len(largest_mesh)}")
